In [144]:
import numpy as np
import pandas as pd
from PIL import Image
from resizeimage import resizeimage
import pickle
import re
import os
import flask
import json
from scipy.misc import imread
from keras.models import load_model
pd.options.display.max_rows = 999

In [145]:
rootpath = "../../egypt/EgyptianHieroglyphDataset/MyTrainTest"
original_dim = (75, 50)
batch_size = 32 # need to be exactly the same as training

In [146]:
tr_rootpath = os.path.join(rootpath, "train")
symbol_df = [x for x in os.listdir(tr_rootpath) if re.search(r"UNKNOWN|^\.", x) == None]
symbol_df += ['M20'] # manual fix
symbol_df = pd.DataFrame(sorted(symbol_df))
symbol_df.reset_index(inplace = True)
symbol_df.columns = ["symbol_num", "symbol"]
symbol_df.tail(1)

,symbol_num,symbol
170,170,Z7


In [147]:
cond = symbol_df["symbol_num"] == 144
symbol_df[cond]

,symbol_num,symbol
144,144,V22


In [135]:
try:
    with open("../models/translate.h5", "rb") as picklefile:
        lexicon = pickle.load(picklefile)
except:
    "Need to run get translation.ipynb!"
lexicon.tail(1)

,symbol,phonetic,translation,freq,freq_cat
6626,O17,tAyty,the shrouded one (of Osiris),0.0,Rare


### merge dict with translation

In [136]:
symbol_df = symbol_df.merge(lexicon, on = "symbol", how = "left")
print(symbol_df.shape)
# symbol_df["freq_cat"].value_counts()
symbol_df.tail(1)

(231, 6)


,symbol_num,symbol,phonetic,translation,freq,freq_cat
230,170,Z7,W,same as G43*,1051.0,Very common


### manually clean with dups 
Note: need much more domain knowledge to do this right, right now it's just a working version  
Also, there are symbols that only appear along with others. I am excluding them here for simplicity

In [137]:
ct = pd.DataFrame(symbol_df["symbol"].value_counts())
ct.reset_index(inplace = True)
ct.columns = ["symbol", "n_dup"]
dup_cond = ct["n_dup"] != 1
dup = ct[dup_cond]
print(dup.shape[0], "symbols duplicated")
dup = dup.merge(lexicon, how = "left", on = "symbol")
# dup

40 symbols duplicated


In [138]:
cond = symbol_df["symbol"].map(lambda x: x in list(ct.loc[~dup_cond, "symbol"]) )
single_rec = symbol_df[cond]
del single_rec["symbol_num"], single_rec["freq"]

In [139]:
# more manual cleanup
single_rec = single_rec.copy()
s34 = single_rec["symbol"] == "S34"
single_rec.loc[s34, "phonetic"] = "anx or ankh"
single_rec.loc[s34, "translation"] = "life, swear, oath"

In [140]:
dedup_rec = pd.read_csv("../models/clean translation.csv")
del dedup_rec["pick_me"]

In [141]:
final_dict = single_rec.append(dedup_rec)
final_dict = symbol_df[["symbol_num", "symbol"]].drop_duplicates().merge(final_dict, how = "right", on = "symbol")
final_dict.reset_index(drop = True, inplace = True)
final_dict.sort_values("symbol_num", inplace = True)
print(final_dict.shape)
final_dict.tail(1)

(171, 5)


,symbol_num,symbol,phonetic,translation,freq_cat
170,170,Z7,W,not,Very common


In [142]:
final_dict

,symbol_num,symbol,phonetic,translation,freq_cat
0,0,A55,XAt,corpse,Rare
1,1,Aa15,NaN,NaN,NaN
2,2,Aa26,NaN,NaN,NaN
3,3,Aa27,NaN,NaN,NaN
4,4,Aa28,NaN,NaN,NaN
5,5,D1,NaN,NaN,NaN
6,6,D10,wDAt,the uninjured eye of Horus,Rare
7,7,D156,NaN,NaN,NaN
8,8,D19,NaN,NaN,NaN
9,9,D2,NaN,NaN,NaN


### pickle dict

In [143]:
with open("../models/dict.pkl", "wb") as pfile:
    pickle.dump(final_dict, pfile)